In [4]:
import  os

In [5]:
%pwd

'd:\\Python\\End-To-End Ml Project\\End-to-end-ML-Project-with-MLflow\\research'

In [6]:
os.chdir("../")

In [7]:
%pwd

'd:\\Python\\End-To-End Ml Project\\End-to-end-ML-Project-with-MLflow'

In [8]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/mspatke/End-to-end-ML-Project-with-MLflow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "mspatke"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "8518bb307abed9989560d016d935f9a3af3abb4a"

In [9]:
#create entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name:Path
    target_column : str
    mlflow_uri : str


In [24]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json


class ConfigurationManager:

      def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH, # its reading from Config_file_path defined in constants
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            self.schema = read_yaml(schema_filepath)


            create_directories([self.config.artifacts_root])

    # reads the model evaluation configuration parameter and return as a object
      def get_model_evaluation_config(self) -> ModelEvaluationConfig:
            
            config = self.config.model_evaluation
            params = self.params.ElasticNet
            schema = self.schema.TARGET_COLUMN

            create_directories([config.root_dir])

            model_evaluation_config = ModelEvaluationConfig(
                  
                  root_dir= config.root_dir,
                  test_data_path= config.test_data_path,
                  model_path= config.model_path,
                  all_params= params,
                  metric_file_name=config.metric_file_name,
                  target_column = schema.name,
                  mlflow_uri = "https://dagshub.com/mspatke/End-to-end-ML-Project-with-MLflow.mlflow"
            )

            return model_evaluation_config

In [11]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

[2023-11-04 23:44:25,810: INFO: utils: NumExpr defaulting to 8 threads.]


In [20]:
class ModelEvalaution:

    def __init__(self, config:ModelEvaluationConfig):

        self.config = config


    def eval_metrics(self, actual, pred):

        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae= mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    

    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis =1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)
            
            (rmse, mae,r2) = self.eval_metrics(test_y, predicted_qualities)

            #saving metric offline
            scores = {"rmse": rmse, "mae": mae, "r2":r2}
            save_json(path = Path(self.config.metric_file_name), data = scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse",rmse)
            mlflow.log_metric("mae",mae)
            mlflow.log_metric("r2",r2)


            if tracking_url_type_store !="file":
                mlflow.sklearn.log_model(model, "model",registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model , "model")


In [25]:
#create pipeline

try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvalaution(config = model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2023-11-05 00:36:05,745: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-05 00:36:05,748: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-05 00:36:05,753: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-11-05 00:36:05,754: INFO: common: created directory at artifacts]
[2023-11-05 00:36:05,755: INFO: common: created directory at artifacts/model_evaluation]
[2023-11-05 00:36:11,082: INFO: common: json file saved at artifacts\model_evaluation\metrics.json]


Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2023/11/05 00:36:34 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticnetModel, version 5
Created version '5' of model 'ElasticnetModel'.
